In [1]:
import hashedEmbeddingBag
import torch 
import torch.nn as nn 
import numpy as np 

In [2]:
MODE="mean"

In [3]:
uma_size = 100
n1 = 100
m1 = 16
_weight = nn.Parameter(torch.randn((uma_size,)).cuda(0), requires_grad=True)
e1 = hashedEmbeddingBag.HashedEmbeddingBag(n1, m1, _weight=_weight, val_offset=0, mode=MODE).cuda(0)



RandomNumbers:  tensor([2038074743,  634329019, 1825252241,  871205356,   80759397])
Central weight rand_hash val_offset 0
HashedEmbeddingBag:  100 16 mode mean hmode rand_hash kmode keymode_hashweight central True key_bits 4 keys_to_use 8 weight_size 100 uma_chunk_size 1


In [4]:
# Create exact replica in nn.Embedding

actual_wt = torch.clone(e1.weight.data)
_weight = torch.arange(uma_size,).float().cuda(0)
e1.weight.data[:] = _weight
indices_all = torch.arange(100).cuda(0)
offsets_all = torch.arange(100).cuda(0)
IDX = e1(indices_all)
e1.weight.data[:] = actual_wt
e_orig = torch.nn.EmbeddingBag(n1, m1, mode=MODE).cuda(0)
e_orig.weight.data[:,:] = e1(indices_all)

In [5]:
indices = torch.arange(25).cuda(0)
offsets = torch.arange(4).cuda(0) * 5

In [6]:
indices

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], device='cuda:0')

In [7]:
offsets

tensor([ 0,  5, 10, 15], device='cuda:0')

In [8]:
# Forward comparision
orig_output = e_orig(indices, offsets)
rz_output = e1(indices, offsets)

In [9]:
print("Forward", torch.max(torch.abs(orig_output - rz_output)))

Forward tensor(0., device='cuda:0', grad_fn=<MaxBackward1>)


In [10]:
orig_loss = torch.sum(orig_output)
rz_loss = torch.sum(rz_output)

In [11]:
orig_loss.backward()
rz_loss.backward()

In [12]:
rz_grad = torch.clone(e1.weight.grad.data)
actual_grad = torch.clone(e_orig.weight.grad.data)

In [13]:
actual_wt_grad = torch.zeros_like(e1.weight).cuda(0)
actual_wt_grad.scatter_add_(0, IDX.long().reshape(-1), actual_grad.reshape(-1))
print("Backward", torch.max(torch.abs(rz_grad - actual_wt_grad)))

Backward tensor(1.1921e-07, device='cuda:0')


In [14]:
actual_wt_grad

tensor([0.3000, 1.0000, 0.3000, 0.8000, 0.2000, 1.0000, 0.2000, 1.0000, 0.4000,
        1.0000, 0.3000, 0.8000, 0.4000, 1.0000, 0.5000, 0.8000, 0.6000, 0.8000,
        0.5000, 0.8000, 0.9000, 0.4000, 0.6000, 0.5000, 0.6000, 0.3000, 0.7000,
        0.4000, 0.7000, 0.2000, 0.9000, 0.3000, 1.0000, 0.4000, 1.0000, 0.4000,
        0.8000, 0.4000, 1.2000, 0.5000, 0.8000, 0.6000, 0.8000, 0.5000, 0.8000,
        0.9000, 0.7000, 0.6000, 0.5000, 0.8000, 0.5000, 0.9000, 0.4000, 0.9000,
        0.4000, 0.8000, 0.5000, 1.2000, 0.3000, 0.8000, 0.4000, 1.0000, 0.4000,
        0.8000, 0.7000, 0.8000, 0.5000, 0.8000, 0.7000, 0.8000, 0.7000, 0.6000,
        0.8000, 0.4000, 0.7000, 0.7000, 0.9000, 0.4000, 0.8000, 0.4000, 0.8000,
        0.2000, 1.2000, 0.4000, 0.8000, 0.3000, 1.0000, 0.3000, 1.0000, 0.4000,
        0.8000, 0.4000, 0.6000, 0.6000, 0.6000, 0.6000, 0.4000, 0.7000, 0.4000,
        0.6000], device='cuda:0')